In [2]:
#code

import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


#This function returns 4 points 
# (lx1,ly1)----Left Line----(lx2,ly2)
# (rx1,ry1)---Right Line----(rx2,ry2)
def find_right_left_lines(img,lines):
    #Threshold the slopes of all lines in the arrgument
    filtered_slopes = []
    filtered_lines = []
    right_lines= []
    left_lines= []
    for line in lines:
        x1,y1,x2,y2 = line[0]
        if x2-x1 == 0:
            slope = 1000;
        else:
            slope = (y2 - y1)/(x2 - x1)
        if abs(slope) > 0.5:
            filtered_slopes.append(slope)
            filtered_lines.append(line) 

    for i,line in enumerate(filtered_lines):
        x1,y1,x2,y2 = line[0]
        if filtered_slopes[i] > 0 and x1 > (img.shape[1] / 2) and x2 > (img.shape[1] / 2):
            right_lines.append(line)
        elif filtered_slopes[i] < 0 and x1 < (img.shape[1] / 2) and x2 < (img.shape[1] / 2):
            left_lines.append(line)  

    #Accumilate all the X and Y coordinates into tow arrays 
    #Run Linear regression to find the m and b (y=mx+b)        
    right_lines_x = []
    right_lines_y = []
    #append Right Lanes
    for line in right_lines:
        x1, y1, x2, y2 = line[0]
        right_lines_y.append(y1)
        right_lines_x.append(x1)
        right_lines_y.append(y2)
        right_lines_x.append(x2)
 
    if len(right_lines_x) > 0:
        r_m, r_b = np.polyfit(right_lines_x, right_lines_y, 1) 
    else:
        r_m, r_b = 1, 1
        
    left_lines_x = []
    left_lines_y = []
    #Append Left Lanes
    for line in left_lines:
        x1, y1, x2, y2 = line[0]
        left_lines_y.append(y1)
        left_lines_x.append(x1)
        left_lines_y.append(y2)
        left_lines_x.append(x2)
        
    if len(left_lines_x) > 0:
        l_m, l_b = np.polyfit(left_lines_x, left_lines_y, 1) 
    else:
        l_m, l_b = 1, 1

    y1 = img.shape[0]
    y2 = 320
    
    rx1 = (y1 - r_b) / r_m
    rx2 = (y2 - r_b) / r_m
    
    lx1 = (y1 - l_b) / l_m
    lx2 = (y2 - l_b) / l_m
    
   
    y1 = int(y1)
    y2 = int(y2)
    rx1 = int(rx1)
    rx2 = int(rx2)
    lx1 = int(lx1)
    lx2 = int(lx2)
    
    # (lx1,ly1)----Left Line----(lx2,ly2)
    # (rx1,ry1)---Right Line----(rx2,ry2)
    final_lines=[(lx1,y1),(lx2,y2),(rx1,y1),(rx2,y2)]
    return final_lines



def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    final_lines=find_right_left_lines(img,lines)
    # (lx1,ly1)----Left Line----(lx2,ly2)
    # (rx1,ry1)---Right Line----(rx2,ry2)
    #Draw Left Lane
    cv2.line(img, final_lines[0], final_lines[1], color, thickness)
    #Draw Right Lane
    cv2.line(img, final_lines[2], final_lines[3], color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    #Check if lines is valid 
    if len(lines) == 0:
        return img
    else:
        draw_lines(line_img, lines,[255,0,0],10)
        return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def process_image(image):
    input_image=image.copy()
    Log_pipeline=0
    imshape=image.shape
    image_gray=grayscale(image)
    if Log_pipeline>0:
        plt.imshow(image_gray,cmap='gray')
        plt.savefig("Write_up_attachments/image_gray1.png")
        plt.show()


    image_gray_blur=gaussian_blur(image_gray,7)
    if Log_pipeline>0:
        plt.imshow(image_gray_blur,cmap='gray')
        plt.savefig("Write_up_attachments/image_gray_blur2.png")
        plt.show()
                     

    image_canny=canny(image_gray_blur, 50, 150)
    if Log_pipeline>0:
        plt.imshow(image_canny,cmap='gray')
        plt.savefig("Write_up_attachments/image_canny3.png")
        plt.show()

    vertices = np.array([[(0,imshape[0]),(450, 300), (490, 300), (imshape[1],imshape[0])]], dtype=np.int32)

    image_ROI=region_of_interest(image_canny,vertices)
    if Log_pipeline>0:
        plt.imshow(image_ROI,cmap='gray')
        plt.savefig("Write_up_attachments/image_ROI4.png")
        plt.show()

    line_img=hough_lines(image_ROI,2, np.pi/180, 15,10, 20)
    if Log_pipeline>0:
        plt.imshow(line_img)
        plt.savefig("Write_up_attachments/image_houg_lines5.png")
        plt.show()
    output_image=weighted_img(line_img,input_image,α=1, β=1, λ=0.)
    if Log_pipeline>0:
        plt.imshow(output_image)
        plt.savefig("Write_up_attachments/image_Weighted6.png")
        plt.show()
    return output_image

def process_images(folder_name):
    Input_Images_list=os.listdir(folder_name)
    #print(Input_Images_list)
    for input_image_name in Input_Images_list:
        print("working on image :" + folder_name+input_image_name)
        image = mpimg.imread(folder_name+input_image_name)
        output_image=process_image(image)
        plt.imshow(output_image)
        mpimg.imsave("test_images_output/op_"+input_image_name,output_image)
        print("writing output to:"+"test_images_output/op_"+input_image_name)
        plt.show()
def process_videos(folder_name):     
    #processing starts Here        
    Input_videos_list=os.listdir(folder_name)
    #print(Input_Images_list)
    for input_video_name in Input_videos_list:        
        white_output = "test_videos_output/" + input_video_name
        ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
        ## To do so add .subclip(start_second,end_second) to the end of the line below
        ## Where start_second and end_second are integer values representing the start and end of the subclip
        ## You may also uncomment the following line for a subclip of the first 5 seconds
        ##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
        clip1 = VideoFileClip("test_videos/"+input_video_name)
        white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
        %time white_clip.write_videofile(white_output, audio=False)
        
#process_images("test_images/")
process_videos("test_videos/")

OSError: [WinError 6] The handle is invalid